Entity resolution and data modeling
-----

In [1]:
reset -fs

In [2]:
from collections import defaultdict, Counter
from pprint import pprint

from fuzzywuzzy import fuzz
import pandas as pd

In [3]:
# Load data 
path = "./data/"
filename = "whd_whisard.csv"

try:
    df_raw = pd.read_csv(path+filename)
except OSError:
    import zipfile
    with zipfile.ZipFile(path+"whd_whisard_20160611.csv.zip", "r") as zf:
        zf.extractall(path)
    df_raw = pd.read_csv(path+filename)

names_raw = df_raw.trade_nm.values  # 2 options for names: trade_nm or legal_name

/Users/brian/anaconda/envs/py3_dl/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2705: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_raw.head(1)

,case_id,trade_nm,legal_name,street_addr_1_txt,cty_nm,st_cd,zip_cd,naic_cd,naics_code_description,case_violtn_cnt,...,flsa_smwsl_bw_atp_amt,flsa_smwsl_ee_atp_cnt,eev_violtn_cnt,h2b_violtn_cnt,h2b_bw_atp_amt,h2b_ee_atp_cnt,sraw_violtn_cnt,sraw_bw_atp_amt,sraw_ee_atp_cnt,ld_dt
0,1513023,Anid Care Home,"Anid Care Home, Inc.",408 Hill Street,Ionia,MI,48846.0,623990,Other Residential Care Facilities,3,...,0.0,0,0,0,0.0,0,0,0.0,0,2015-02-20 01:00:06.112241


In [5]:
def normalize_name(name_raw):
    "Given entity name from data, return clean (semi-normalized) name for entity matching"
    name_normalized = name_raw.strip().lower()
    return name_normalized

def check_entity(entities, name):
    "Look for fuzzy name matching in entities"
    if not name in nonentities:
        return name
    else:
        return "NOT VALID ENTITY: "+name

In [6]:
# entities = {entity_id: {alais_1, alais_2}}
entities = defaultdict(set) 
entities[-1].update(['07-Jan']) # Nonentities
entities[1003].update(["USPS", 'United States Postal Service', 'US Postal Service', 'U.S. Postal Service'])
entities[1004].update(['Super 8', 'Super 8 Motel'])
entities[1005].update(["KFC", "Kentucky Fried Chicken"])

ratio_threshold = 90
assert fuzz.token_sort_ratio("McDonald's", 'McDonalds') > ratio_threshold

In [7]:
entities

defaultdict(set,
            {-1: {'07-Jan'},
             1003: {'U.S. Postal Service',
              'US Postal Service',
              'USPS',
              'United States Postal Service'},
             1004: {'Super 8', 'Super 8 Motel'},
             1005: {'KFC', 'Kentucky Fried Chicken'}})

In [8]:
Counter(names_raw).most_common()[:10]

[('Subway', 932),
 ('Dunkin Donuts', 338),
 ("McDonald's", 290),
 ('Holiday Inn Express', 214),
 ('Days Inn', 208),
 ('McDonalds', 197),
 ('USPS', 186),
 ('Burger King', 171),
 ('United States Postal Service', 170),
 ('Dairy Queen', 163)]

In [ ]:
# Fuzzy check for name in know alaises. Asign unique id. Update list of know alaises
entityid_name_check = []

for name in names_raw:
    try: 
        entityid = next(k for k,alaises in entities.items() 
                            for entity in alaises if fuzz.token_sort_ratio(name, entity) > ratio_threshold)
    except StopIteration: # If entity is not in alias, add
        entityid = max(entities.keys())+1
    entities[entityid].add(name)
    entityid_name_check.append((entityid, name))

In [ ]:
pprint(entities)

In [ ]:
entityid_name_check

In [ ]:
# Deterministic check and add
for name in names_raw[:10]:
    if not [k for k,v in entities.items() if name in v]: # If entity is not in alias, add
            entities[max(entities.keys())+1] = name